<br>

# Técnicas Matemáticas para Big Data - Project NN?
<br><br>


GROUP NN:
- Student 1 - Nº 106078 - 33.33% Work Participation
- Student 2 - Nº xxxxx - ??% Work Participation
- Student 3 - Nº xxxxx - ??% Work Participation

<br><br>

## 1. Introduction to the problem of study [1,0 valor]

### Estimating Unique Customers with the HyperLogLog Algorithm

We aim to process e-commerce purchase data chronologically, simulating a **data stream** using real sales data obtained from the [Online Retail Dataset – UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/352/online+retail).

The dataset contains all transactions that occurred between **01/12/2010 and 09/12/2011** for a **UK-based non-store online retailer**.

Each new event (purchase) is added to the data stream, ordered by its corresponding **timestamp**.  
The main objective is to **estimate the number of unique customers** who made a purchase in real time.

With continuous technological advancements and the ever-growing volume of users and data, companies must monitor customer activities in **real time**. However, the massive amount of data generated requires both **high processing capacity** and **speed**.  

Processing data on a large scale is a current technological challenge, as it demands the ability to deliver accurate statistics, analytics, and insights almost instantaneously, without compromising system performance or precision.

In this context, the problem addressed focuses on **estimating the number of unique buyers** in an online sales platform as new purchase events continuously arrive.  
Each transaction in the dataset represents a sale made by a specific customer at a given moment, allowing us to simulate a **data stream** where events are processed in chronological order.

The goal is to estimate, in real time, **how many distinct customers have made purchases** up to any given point.


<br><br>
## 2. Brief and general description of the approach and methods used [1,5 valor]


### Motivation

Processing massive amounts of data by storing them in a database and iterating over each record would require **significant computational resources**, making such an approach infeasible.

In a traditional setup, counting distinct customers would require storing every `CustomerID` in a database or cache and performing periodic scans as new records arrive.  
Although this approach provides exact results, it becomes **inefficient** when applied to large-scale datasets.

The amount of memory required grows **linearly** with the number of customers, and recomputing distinct counts from stored data can become **very expensive** in terms of both time and system resources.

### Solution: The HyperLogLog Algorithm (HLL)

To perform this task efficiently, we implemented the **probabilistic algorithm [HyperLogLog(HLL)](https://en.wikipedia.org/wiki/HyperLogLog)**, designed to **estimate the cardinality** (number of distinct elements) in large datasets.

The HyperLogLog algorithm can estimate cardinalities on the order of 10⁹ elements with a **relative error below 2%**, while using only about **1.5 kB of memory**.

### Methodology

In **real-time analytics systems**, where millions of events may arrive every minute, storing and maintaining exact counts of unique elements quickly becomes impractical.

Instead of storing every individual customer identifier, HyperLogLog(HLL):

1. **Applies a hash function** to each incoming `CustomerID` into a value;  
2. **Updates a set of registers** represent the overall distribution of those hashes.

This let the algorithm estimate the **number of unique costumers** observed so far, with a small and controllable margin of error determined by the **precision parameter `p`**.

### Advantages

The main advantage of HyperLogLog lies in its **memory efficiency** and **processing speed**:

- Uses a **fixed amount of memory**, regardless of the stream size;  
- Processes each incoming event in **constant or near-constant time**;  
- Maintains **high accuracy with minimal computational cost**.


<br><br>
## 3. Brief History and literature review of the problem and methods/algorithms [1,5 valor]

The problem of efficiently counting distinct elements in large datasets has long been a central challenge in data processing and database systems.  
Early approaches relied on exact methods, which required storing all unique identifiers in memory or on disk.  
While these techniques are straightforward, their **space complexity grows linearly** with the number of unique elements, making them impractical for large-scale or streaming environments where data arrives continuously and in massive volumes.

In the early 1980s, **Philippe Flajolet and G. Nigel Martin** introduced the [Flajolet–Martin (FM) algorithm](https://algo.inria.fr/flajolet/Publications/src/FlMa85.pdf), one of the first probabilistic algorithms for approximate distinct counting.  
Their method relied on hashing each element and observing the position of the least significant 1-bit in the binary representation of the hash values.  
The expected position of this bit could be used to estimate the logarithm of the cardinality of the dataset.  
Although revolutionary, the original FM algorithm exhibited high variance and limited accuracy for large datasets.
This original algorithm used only a global counter to store the highest value of the first bit position, 1, that is, the highest number of consecutive leading zeros observed in the hashes.
The problem is that this variable had a very high variance. Therefore, just a single extreme value can drastically change the result.


To improve the precision of probabilistic counting, **Flajolet et al.** later proposed the **LogLog** algorithm and, subsequently, the [HyperLogLog (HLL)](https://algo.inria.fr/flajolet/Publications/FlFuGaMe07.pdf) algorithm in 2007.  
The HyperLogLog introduced a refined estimator based on the **harmonic mean** of multiple sub-estimators (called registers) and incorporated a **bias-correction constant** through the concept of **stochastic averaging**.  
This design reduced the **relative standard error (RSE)** to approximately \(1.04 / \sqrt{m}\), where \(m = 2^p\) is the number of registers.  
Instead of maintaining a single global counter, the hash space is divided into \(m\) disjoint subsets (or buckets), and an independent estimate is computed for each subset.

The HyperLogLog algorithm thus achieved an elegant balance between **accuracy**, **computational efficiency**, and **memory usage**.

HLL thus provided an elegant balance between **accuracy**, **computational efficiency**, and **memory usage**.

Over time, the HyperLogLog algorithm became one of the most widely adopted techniques for large-scale analytics.  
It is currently implemented in major data systems such as **Google BigQuery**, **Redis**, **Apache Spark**, and **PostgreSQL**, where it supports real-time analytics and streaming queries.  
Its **merge** property — the ability to combine multiple estimators into a single global one — makes it particularly suitable for **distributed architectures** and **parallel computation** environments.

Beyond web analytics and online systems, HyperLogLog is used in **network monitoring**, **fraud detection**, **ad impression counting**, and **data deduplication**.  
Its efficiency lies in providing **near-constant memory usage** and **fast incremental updates**, allowing accurate estimation of distinct elements in data streams that would otherwise be too large to store or process exactly.

HyperLogLog represents a key milestone in the evolution of probabilistic counting algorithms, combining solid mathematical foundations with proven scalability and practical relevance in modern Big Data applications.


<br><br>
## 4. About the main method/algorithm used [1,5 valor]



### Core principle

The algorithm uses hash functions to obtain a binary representation of the data and the key insight behind HyperLogLog is that the position of the **first 1-bit** in the representation of a uniformly distributed hash value carries information about the magnitude of the dataset’s cardinality.  
If the dataset is large, it becomes increasingly likely to observe hash values with long sequences of leading zeros.  
By analyzing the distribution of these positions over many independent hash samples, the algorithm can infer the total number of unique elements.

### Data structure

HyperLogLog maintains an array of \( m = 2^p \) **registers**(buckets), each of which stores a integer that represents the maximum number of leading zeros observed among the hash values assigned to that register.  
The parameter \( p \) determines both the **number of registers** and the **precision** of the estimation:
- A higher \( p \) means more registers (greater accuracy) but slightly higher memory usage.
- A lower \( p \) reduces memory but increases the standard error.

The **relative standard error (RSE)** of the algorithm is approximately:

\[
\text{RSE} = \frac{1.04}{\sqrt{m}} = \frac{1.04}{\sqrt{2^p}}
\]

Thus, each increment of one bit in \( p \) reduces the error by roughly \( \sqrt{2} \).

### Algorithm procedure

1. **Hashing:**  
   Each element \( x \) is passed through a hash function (e.g., SHA-1 or SHA-256) to produce a 64-bit value \( h(x) \).  
   This ensures a uniform and random distribution across the hash space.

2. **Register selection:**  
   The first \( p \) bits of the hash are used to select which register \( j \) to update.  
   This splits the hash space into \( 2^p \) buckets.

3. **Zero counting:**  
   The remaining bits of the hash are used to count the number of leading zeros \( \rho(x) \) after the prefix.  
   The corresponding register \( M[j] \) is updated as:

   \[
   M[j] = \max(M[j], \rho(x))
   \]

4. **Estimation:**  
   After all elements are processed, the algorithm computes the harmonic mean of the register values:

   \[
   E = \alpha_m \cdot m^2 \cdot \left( \sum_{j=1}^{m} 2^{-M[j]} \right)^{-1}
   \]

   where \( \alpha_m \) is a **bias correction constant** that depends on \( m \):
   - \( \alpha_{16} = 0.673 \)
   - \( \alpha_{32} = 0.697 \)
   - \( \alpha_{64} = 0.709 \)
   - For \( m \geq 128 \): \( \alpha_m = 0.7213 / (1 + 1.079/m) \)

### Corrections and refinements

The HyperLogLog algorithm applies different estimation formulas depending on the observed cardinality range.  
This prevents systematic bias that may occur when the dataset is either too small or too large relative to the number of registers.  
The algorithm therefore operates under **three distinct regimes**:

1. **Small-range correction (Linear Counting):**  
   When the raw estimate \( E \) is **less than \( 2.5m \)**, the HyperLogLog tends to overestimate due to a large number of empty registers.  
   In this regime, the algorithm switches to **linear counting**, using the formula:

   \[
   E' = m \cdot \ln\left(\frac{m}{V}\right)
   \]

   where \( V \) is the number of registers with value 0.  
   This correction provides more accurate results when the number of distinct elements is small compared to the number of registers.

2. **Intermediate-range (Standard Estimation):**  
   When the cardinality is within a normal operating range — typically between \( 2.5m \) and \( (1/30) \times 2^{32} \) —  
   the **raw HyperLogLog estimate** is sufficiently accurate and no correction is required:

   \[
   E' = E = \alpha_m \cdot m^2 \cdot \left( \sum_{j=1}^{m} 2^{-M[j]} \right)^{-1}
   \]

   This is the most common regime in practical applications, where the estimator achieves its theoretical relative standard error of \( 1.04 / \sqrt{m} \).

3. **Large-range correction (Bias Reduction):**  
   When the estimated value \( E \) exceeds approximately \( (1/30) \times 2^{32} \),  
   the algorithm may **underestimate** due to hash collisions approaching the 32-bit space limit.  
   In this case, a **large-range correction** is applied:

   \[
   E' = -2^{32} \cdot \ln\left(1 - \frac{E}{2^{32}}\right)
   \]

   This compensates for the nonlinearity that arises when the hash space becomes saturated and the probability of collisions increases.

These three regimes together ensure that HyperLogLog remains accurate across a wide range of dataset sizes — from a few hundred distinct elements to billions — using the same fixed memory footprint.  
This adaptability is one of the main reasons why HyperLogLog has become the de facto standard for scalable cardinality estimation in data stream processing.


### Merge property

A major advantage of HyperLogLog is its **mergeability**.  
Given two HLLs \( A \) and \( B \) built from different subsets of the data, their union can be computed simply as:

\[
M_{\text{merged}}[j] = \max(M_A[j], M_B[j])
\]

This operation is associative and idempotent, allowing partial counts from distributed systems to be combined efficiently without double-counting elements.  
This makes HyperLogLog highly suitable for **parallel** and **distributed architectures**, such as those used in real-time analytics and Big Data pipelines.

### Complexity and memory efficiency

The algorithm runs in **O(1)** time per insertion and requires **O(2^p)** memory, which is constant with respect to the total number of elements processed.  
Typical configurations (e.g., \( p = 14 \)) require only a few kilobytes of memory and achieve a relative error below 1%.  
This balance between accuracy, speed, and memory efficiency is what makes HyperLogLog one of the most practical algorithms for streaming distinct counting tasks.


<br><br>

## 5. Python imports and global configurations [0,5 valor]

### Install and import the necessary libraries to compute the Bayesian Network and perform other methods  

In [9]:


%pip install pandas
%pip install matplotlib
%pip install ucimlrepo

import hashlib
import matplotlib
import math
from ucimlrepo import fetch_ucirepo


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


<br><br>

## 6. Dataset and variables explanation [1,5 valor]

The dataset used in this project is the **Online Retail Dataset** from the UCI Machine Learning Repository.  
It contains **541,910 transactions** recorded by a United Kingdom–based online gift retailer between **December 2010 and December 2011**.  
Each record represents an individual product purchased within an invoice, providing detailed information about sales activity across multiple countries and customers.

### Structure of the dataset

The dataset includes the following main variables:

| Variable | Description |
|-----------|-------------|
| **InvoiceNo** | Unique identifier for each invoice. Cancellations are indicated by invoices starting with the letter “C”. |
| **StockCode** | Unique code assigned to each product. |
| **Description** | Text description of the product. |
| **Quantity** | Number of units purchased (negative values indicate product returns). |
| **InvoiceDate** | Timestamp of the transaction (date and time when the invoice was issued). |
| **UnitPrice** | Price per unit of the product in pounds sterling (£). |
| **CustomerID** | Unique identifier for the customer who made the purchase. |
| **Country** | Country of the customer. |

### Data preparation

Before the data can be used for streaming analysis, several preprocessing steps are applied:

1. **Removal of missing values:**  
   Some transactions lack a `CustomerID`.  
   These rows are excluded since the study focuses on identifying **unique buyers**, and missing identifiers cannot be used for distinct counting.

2. **Filtering valid sales:**  
   Records with negative `Quantity` correspond to product returns.  
   Although these events may be relevant in other analyses, they do not represent a new purchase.  
   Thus, only transactions with `Quantity > 0` are kept to represent genuine purchase events.

3. **Sorting by time:**  
   The dataset is sorted by `InvoiceDate` to simulate a **chronological data stream**, where each transaction is processed in the order it occurred.

4. **Type normalization:**  
   The `CustomerID` is treated as a string to ensure consistency.

5. **Feature selection:**  
   For this study, only the columns `InvoiceDate`, `CustomerID`, and `Country` are required.  
   Other variables (e.g., `StockCode`, `UnitPrice`) are kept only for potential extensions, such as analyzing unique products or merging HLLs across regions.

### Motivation for using this dataset

The Online Retail dataset is useful for this project for several reasons:

- It provides **real-world transactional data**, where each row naturally represents a **streaming event**.  
- The presence of **timestamps** (`InvoiceDate`) enables the simulation of a **real-time data stream**, allowing HyperLogLog to be tested in an online setting.  
- The inclusion of multiple countries also allows demonstration of HLL’s **merge** property — combining partial estimators built per region into a global one.

### 6.4. Variables used in this study

The main variable analyzed is **`CustomerID`**, which identifies each buyer.  
The goal is to estimate, in real time, the number of **unique customers** who have made purchases up to any given point in time.

Other variables are used for contextual purposes:
- **`InvoiceDate`** is used to order the transactions chronologically, forming the basis of the data stream.  
- **`Country`** allows the study of customer diversity across geographic regions and serves as a grouping variable for distributed HLL estimation.


<br><br>

## 7. Main code as possible solution to the problem [1,5 valor] 

**Hashing**  
Each element is processed through a deterministic hash function (`sha1` or 
`sha256`). Only the first 64 bits of the hash are used for performance and 
consistency.

**Register Selection**  
The first *p* bits of the hash determine which register (bucket) is updated.  
This means the structure has `m = 2^p` registers.

**Leading Zero Count (Rho Value)**  
The remaining bits of the hash are inspected to count the number of leading
zeros, plus one:

$$
\rho = \text{count leading zeros}(w) + 1
$$

This value statistically represents how "rare" a hash position is.

**Register Update Rule**  
Registers store **only the maximum** rho value observed for their index:

$$
M[j] = \max(M[j], \rho)
$$

**Harmonic Mean Estimation**  
After processing the stream, the estimate is computed using:

$$
E = \alpha_m \cdot \frac{m^2}{\sum_{j=1}^{m} 2^{-M[j]}}
$$

This formula uses the **harmonic mean** of the bucket contributions and the 
empirical correction factor:

$$
\alpha_m = 
\begin{cases}
0.673 & m = 16 \\
0.697 & m = 32 \\
0.709 & m = 64 \\
\frac{0.7213}{1 + 1.079/m} & m \ge 128
\end{cases}
$$

**Small Range Correction**  
For datasets where the estimate is relatively small and many registers remain 
empty, the linear counting correction improves accuracy:

$$
E' = m \cdot \ln\left(\frac{m}{V}\right)
\quad \text{if } E \le 2.5m
$$

Where `V` is the number of registers with value zero.

**Final Output**  
The method `estimate()` returns a floating-point approximation.  
The method `count()` provides the rounded integer cardinality.


In [11]:
class HyperLogLog:
    
    def __init__(self, p=10, hash_name= "sha1"):
        self.p = p
        self.m = 2**p
        self.M = [0] * self.m  # registers
        self.alpha_m = self._alpha(self.m)
        self.hash_name = hash_name
    
    @staticmethod
    def _alpha(m):
        if m == 16:  return 0.673
        if m == 32:  return 0.697
        if m == 64:  return 0.709
        # m >= 128
        return 0.7213 / (1.0 + 1.079 / m)
    
    def apply_hash(self, value):
        if not isinstance(value, (bytes, bytearray)):
            value = str(value).encode("utf-8")
        if self.hash_name == "sha1":
            digest = hashlib.sha1(value).digest()
        else:
            digest = hashlib.sha256(value).digest()
        return int.from_bytes(digest[:8], "big")
    
    def process_element(self,item):
        hash = self.apply_hash(item)
        bin_hash = bin(hash)[2:].zfill(64) # Converte para binário (sem '0b') e assegura 64 bits
        indice_bits = bin_hash[:self.p]
        resto_bits = bin_hash[self.p:]
        j = int(indice_bits, 2)
        zeros = len(resto_bits) - len(resto_bits.lstrip("0"))
        rho = zeros + 1  # +1 
        self.M[j] = max(self.M[j], rho) #Atualizar a lista de buckets
        
    def estimate(self):
        m = self.m
        Z = sum((2.0 ** -value) for value in self.M)
        H = m / Z
        E = self.alpha_m * m * H
        E = self.small_range_correction(E)
        return E
    
    def small_range_correction(self, E):
        m = self.m

        if E <= 2.5 * m:
            V = self.M.count(0)
            if V > 0:
                return m * math.log(m / V)

        return E

    
    def count(self):
        return int(round(self.estimate()))

    def process_list(self,list):
        for element in list:
            self.process_element(element)                
    

<br><br>

## 8. Analysis of Example 1 [3,0 valor]

<br><br>

## 9. Analysis of Example 2 [3,0 valor]

<br><br>
## 10. Pros and cons of the approach [2,0 valor]

<br><br>
## 11. Future improvements [2,0 valor]

<br>
<div style="text-align: center;">
    <br><br>
    <p style="font-size: 40px;">References [1,0 valor]</p>
</div>
<br>
